<a href="https://colab.research.google.com/github/github.com/Gan4x4/ml_snippets/blob/main/Training/Lightning.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Базовая задача которую решает Lghtning это реализация train loop

In [ ]:
import torch

model =

# managing device
optimizer = torch.optim.SGD(model.parameters(), lr=lr)  # Weight update
criterion = nn.CrossEntropyLoss()  # Loss function

for epoch in range(epochs):
    correct, total = 0, 0
    for batch in train_loader:
        # Processing one batch
        img, label = batch
        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels.to(device))
        loss.backward()
        # Calclulate metrics
        # Save metrics to logs
        optimizer.step()

     # Validation step
     # Save checkpoint

Using device: cuda


In [ ]:
import lightning.pytorch as pl
from torch import optim
import torchmetrics

class LitCNN(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.metric = torchmetrics.classification.MulticlassAccuracy(10)
        self.criterion = nn.CrossEntropyLoss()

    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        loss = self.criterion(out, y)
        self.log("train_loss", loss)
        self.metric.update(out,y)
        return loss

    def on_train_epoch_end(self):
        self.log("accuracy/train", self.accuracy_train.compute())
        self.accuracy_train.reset()

    def on_validation_epoch_end(self):
        self.log("accuracy/val", self.accuracy_val.compute())
        self.accuracy_val.reset()

    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        self.accuracy_val.update(out, y)

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.model.parameters(), lr=0.001)
        return optimizer

Shadow work


* managing devices
* creating checkpoints
* finding LR

Standartize

* train loop creation
* logging




In [ ]:
import lightning as L
import torchmetrics

class LitBasic(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.m = model
        self.criterion = nn.CrossEntropyLoss()
        self.metric = torchmetrics.Accuracy(task="multiclass", num_classes=10)

    def on_train_epoch_start(self):
        self.metric.reset()
        print("on_train_epoch_start")

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        out = self.m(x)
        loss = self.criterion(out, y)
        self.metric.update(out, y)
        self.log("loss", loss,prog_bar = True)
        return loss

    def on_validation_epoch_start(self):
        print("on_validation_epoch_start")
        self.log("accuracy/train", self.metric.compute(),prog_bar = True)
        self.metric.reset()

    def validation_step(self, batch, batch_idx):
        # this is the validation loop
        x, y = batch
        out = self.m(x)
        self.metric.update(out,y)

    def on_validation_epoch_end(self):
        print("on_validation_epoch_end")
        self.log("accuracy/val", self.metric.compute(),prog_bar = True)
        self.metric.reset()

    def on_train_epoch_end(self):
        print("on_train_epoch_end")

    """          optimizers """
    #def configure_optimizers(self):
    #    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    #    return optimizer

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=0.01)
        return optimizer


# Experiment naming

https://lightning.ai/docs/pytorch/stable/extensions/generated/lightning.pytorch.loggers.TensorBoardLogger.html#lightning.pytorch.loggers.TensorBoardLogger

Logger setup
from lightning.pytorch.loggers import TensorBoardLogger

In [ ]:
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")
trainer = Trainer(logger=logger)

Log two variable in one axxis

In [ ]:
https://stackoverflow.com/questions/66287075/pytorch-lightning-multiple-scalars-e-g-train-and-valid-loss-in-same-tensorbo

# Checkpoint
- переименовывать ключи
https://lightning.ai/docs/pytorch/stable/common/checkpointing_basic.html

In [ ]:
https://lightning.ai/docs/pytorch/stable/common/checkpointing_basic.html

#Learning rate

https://lightning.ai/docs/pytorch/2.1.0/advanced/training_tricks.html#learning-rate-finder

# Test

https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.core.LightningModule.html#lightning.pytorch.core.LightningModule.test_step

https://pytorch-lightning.readthedocs.io/en/1.4.9/common/test_set.html